In [ ]:
#notebook for training models on imagenet and saving them

In [8]:
#imports

import sys
from pathlib import Path

import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn as nn

In [9]:
module_path = str(Path.cwd().parents[0].parents[0] / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

from dataloader import *
import trainers
import nnmodels

ds_train_path = str(Path.cwd().parents[0].parents[0] / "data" / "imageNet" / "ILSVRC" / "Data" / "CLS-LOC" / "train")
ds_test_path = str(Path.cwd().parents[0].parents[0] / "data" / "imageNet" / "ILSVRC" / "Data" / "CLS-LOC" / "test")

Running on the GPU


In [10]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [ ]:
bs = 16
trans = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = datasets.ImageFolder(ds_train_path, trans)
trainloader = DataLoader(train_dataset, bs)

test_dataset = datasets.ImageFolder(ds_test_path, trans)
testloader = DataLoader(test_dataset, bs)

In [ ]:
model = nnmodels.ConvModel(0)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
for epoch in range(5):
    _, _ = trainers.trainLoop(model, trainloader, criterion, optimizer)

In [ ]:
#test block, since testLoop has too many binary speficic actions
testRunningLoss = 0.0
y_pred = []
y_true = []
for j, data in enumerate(testloader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        output = model(inputs.permute(0,3,1,2))# Feed Network

        output1 = (torch.max(output.to(device), 1)[1])
        y_pred.extend(output1) # Save Prediction
        
        y_true.extend(labels) # Save Truth
        loss = criterion(output, labels.type(torch.LongTensor).to(device))
        testRunningLoss += loss.item() * inputs.size(0)
correct = (torch.FloatTensor(y_pred) == torch.FloatTensor(y_true)).sum()
testAccuracy = correct / len(y_true)
testRunningLoss = testRunningLoss/len(y_pred)

In [ ]:
print(testAccuracy)

In [ ]:
class Resnet50(nn.Module):
    def __init__(self, model):
        super(Resnet50, self).__init__()
        self.model = model
        self.my_new_layer = nn.Sequential(nn.Linear(1000, 2))
    
    def forward(self, x):
        x = self.model(x)
        x = self.my_new_layer(x)
        return x

In [ ]:
newModel = OutputLayer(model)
newModel